In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

Inspired by: http://www.scielo.org.za/pdf/sajie/v31n3/08.pdf

In [3]:
items = pd.read_csv("../../Data/items.csv", sep="|")
orders = pd.read_csv("../../Data/orders_before_dec.csv", sep="|")
orders["date"] = pd.to_datetime(orders["date"])

In [4]:
items.head()

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,NaN
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."
3,21399,1090,10,1,511,0,0,[3270]
4,8504,768,4,1,484,0,66,[2470]


In [5]:
orders.head()

,date,userID,itemID,order
0,2020-06-01,38769,3477,1
1,2020-06-01,42535,30474,1
2,2020-06-01,42535,15833,1
3,2020-06-01,42535,20131,1
4,2020-06-01,42535,4325,1


In [6]:
item_orders = pd.merge(orders, items, how="left", on="itemID")[["date", "userID", "itemID"]]
item_orders.head()

,date,userID,itemID
0,2020-06-01,38769,3477
1,2020-06-01,42535,30474
2,2020-06-01,42535,15833
3,2020-06-01,42535,20131
4,2020-06-01,42535,4325


### Feature 1: Days between orders per product

In [8]:
order_basket = item_orders.groupby(["date", "userID"])["itemID"].apply(list).reset_index(name="basket")
order_basket

,date,userID,basket
0,2020-06-01,4,"[18860, 30779]"
1,2020-06-01,20,[18613]
2,2020-06-01,55,"[9547, 17912, 10844, 24763]"
3,2020-06-01,76,"[26645, 23050, 2787]"
4,2020-06-01,89,[6287]
...,...,...,...
255816,2020-11-30,46004,"[19983, 23582, 25802]"
255817,2020-11-30,46058,"[18637, 18630, 17560, 14408, 9058, 24641, 19049]"
255818,2020-11-30,46069,"[29992, 18116, 10849, 19725]"
255819,2020-11-30,46081,"[25034, 91, 32332]"


In [92]:
# Create column containing all buy dates per user-item combination
product_buy_dates = item_orders.groupby(["userID", "itemID"])["date"].apply(list).reset_index()

# Column names for all dates
size = len(max(product_buy_dates["date"], key=len))
columns = [word+str(number) for word, number in zip(["date_"] * size, range(size))]

# Split dates to own columns
split_dates = pd.DataFrame(product_buy_dates["date"].tolist(), columns=columns)

# Merge 
product_buy_dates = pd.concat([product_buy_dates, split_dates], axis=1)

# Drop list column
product_buy_dates = product_buy_dates.drop("date", axis=1)

In [76]:
product_buy_dates.head()

,userID,itemID,date_0,date_1,date_2,date_3,date_4,date_5,date_6,date_7,...,date_13,date_14,date_15,date_16,date_17,date_18,date_19,date_20,date_21,date_22
0,0,1505,2020-09-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,0,9325,2020-11-20,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,0,12468,2020-08-03,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,0,12505,2020-08-18,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,0,15083,2020-08-03,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [193]:
product_buy_diffs = product_buy_dates.copy(deep=True)

for i in range(3, len(product_buy_dates.columns[2:])+2):
    new_column = "t_"+ str(i-3)
    product_buy_diffs[new_column] = round(((product_buy_diffs.iloc[:, i] - product_buy_diffs.iloc[:, i-1]).dt.days) / 7)
    
product_buy_diffs = product_buy_diffs.drop(columns=columns)
product_buy_diffs = product_buy_diffs.fillna(27)


In [195]:
product_buy_diffs.head()

,userID,itemID,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,...,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21
0,0,1505,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
1,0,9325,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
2,0,12468,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
3,0,12505,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
4,0,15083,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0


### Feature 2 : Days since prior order per product

### Further Feature Deriavations

In [229]:
user_item_data =  pd.DataFrame(np.unique(item_orders[["userID", "itemID"]], axis=0), columns=["userID", "itemID"])

user_item_data["max days"] = product_buy_diffs[product_buy_diffs.columns[2:]].max(axis=1)
user_item_data["min days"] = product_buy_diffs[product_buy_diffs.columns[2:]].min(axis=1)
user_item_data["avg_days"] = product_buy_diffs[product_buy_diffs.columns[2:]].mean(axis=1)
user_item_data["variance"] = product_buy_diffs[product_buy_diffs.columns[2:]].var(axis=1)

user_item_data = pd.merge(user_item_data, product_buy_diffs, on=["userID", "itemID"])

In [230]:
user_item_data

,userID,itemID,max days,min days,avg_days,variance,t_0,t_1,t_2,t_3,...,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21
0,0,1505,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
1,0,9325,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
2,0,12468,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
3,0,12505,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
4,0,15083,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667296,46137,2667,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
667297,46137,20209,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
667298,46137,28343,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
667299,46137,28900,27.0,27.0,27.0,0.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
